In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

In [2]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)

def get_summary(df, metric, lower_percentile=0.25, upper_percentile=0.75):
    dfgb = df.groupby(["upgrade_name"])
    dfs = pd.concat([
        dfgb["sample_weight"].count().rename("n_samples"),
        dfgb["sample_weight"].sum().rename("n_represented"),
        dfgb.apply(lambda x: weighted_mean(x, metric)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=lower_percentile)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=upper_percentile)),
    ], axis=1)
    dfs = dfs.sort_values(by=f"{metric} wt_mean", ascending=False)

    return dfs

## Whole Community Upgrade Cost Per Dwelling Unit

In [3]:
def community_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    display(results['upgrade_cost_usd'])
    return results['upgrade_cost_usd']

## Whole Community Total Annual Savings

In [4]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings_1(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>mmbtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')    

# flavor 2 - site, electricity, and non-electricity
def figure_total_savings_2(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>mmbtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_2.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu','saving_energy.electricity_kwh','saving_energy.natural_gas_fuel_oil_propane_mmbtu']]

# flavor 3 - site energy only
def figure_total_savings_3(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

## Whole Community Annual Savings Per Dewlling Unit

In [5]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    display(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu']]

## Segment Upgrade Cost Per Dwelling Unit

In [6]:
def segment_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3 = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4 = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd']/segment3['sample_weight']
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd']/segment4['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    segment3 = segment3.sort_values('upgrade_cost_usd')
    segment4 = segment4.sort_values('upgrade_cost_usd')
    
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd'].round(1)
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    display(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    display(segment2['upgrade_cost_usd'])
    print('segment3: upgrade cost (usd) per dwelling unit')
    display(segment3['upgrade_cost_usd'])
    print('segment4: upgrade cost (usd) per dwelling unit')
    display(segment4['upgrade_cost_usd'])
    return (segment1['upgrade_cost_usd'], segment2['upgrade_cost_usd'], segment3['upgrade_cost_usd'], segment4['upgrade_cost_usd'])

## Segment Savings Per Dollar Invested

In [7]:
def segment_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['upgrade_cost_usd']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    segment3_agg[indicator] = segment3_agg[indicator].round(5)
    segment4_agg[indicator] = segment4_agg[indicator].round(5)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator]) 
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Total Savings

In [8]:
def segment_annual_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)

    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Savings Per Dwelling Unit

In [9]:
def segment_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Percent Stock with Net Positive Savings

In [10]:
def figure_positive_negative_savings(segment, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}.svg')
    return (percentage ['cbill_p'],percentage ['emission_p'],percentage ['energy_p'])

## Segment Average Energy Burden (%) Before and after Upgrade.

In [11]:
def segment_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    segment3_agg['baseline_energy_burden_2023_cbills.%'] = segment3_agg['baseline_energy_burden_2023_cbills.%']/segment3_agg['sample_weight']
    segment4_agg['baseline_energy_burden_2023_cbills.%'] = segment4_agg['baseline_energy_burden_2023_cbills.%']/segment4_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    segment3_agg['energy_burden_diff'] = segment3_agg['baseline_energy_burden_2023_cbills.%'] - segment3_agg[indicator]
    segment4_agg['energy_burden_diff'] = segment4_agg['baseline_energy_burden_2023_cbills.%'] - segment4_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment3_agg = segment3_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment4_agg = segment4_agg.sort_values(['energy_burden_diff'], ascending=False)
    
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
    segment3_agg = segment3_agg[segment3_agg.upgrade_name != 'baseline']
    segment4_agg = segment4_agg[segment4_agg.upgrade_name != 'baseline']
    
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment3_agg = segment3_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment4_agg = segment4_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment3_agg['before'] = segment3_agg['before'].round(1)
    segment4_agg['before'] = segment4_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    segment3_agg['after'] = segment3_agg['after'].round(1)
    segment4_agg['after'] = segment4_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    display(segment1_agg[['before', 'after']])
    print('segment2')
    display(segment2_agg[['before', 'after']])
    print('segment3')
    display(segment3_agg[['before', 'after']])
    print('segment4')
    display(segment4_agg[['before', 'after']])
    
    return(segment1_agg[['before', 'after']],segment2_agg[['before', 'after']],segment3_agg[['before', 'after']],segment4_agg[['before', 'after']])

In [12]:
community = 'louisville'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")
saving_results = pd.DataFrame()

# 1. Results for Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [13]:
results = community_mean_upgrade_cost(community_results)
saving_results['whole_community_upgrade_cost_mean_usd'] = results

upgrade_cost_usd per dwelling unit


upgrade_name
Electric Cooking                                                    1000.8
Electric Clothes Dryer                                              1123.9
Induction Cooking                                                   1720.9
Heat Pump Clothes Dryer                                             2261.5
Heat Pump Water Heater                                              2826.5
Basic Enclosure                                                     5145.5
Enhanced Enclosure                                                  7033.2
Mininum Efficiency Heat Pump with Existing Heat Backup             15391.5
Mininum Efficiency Heat Pump with Electric Heat Backup             15496.4
Mininum Efficiency Whole Home Electrification                      18101.8
High Efficiency Heat Pump with Electric Heat Backup                24498.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        29028.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     29895.9
High Efficie

## 1.2 Total Annual Savings

In [14]:
figure_total_savings_1(community_results)

In [15]:
results = figure_total_savings_2(community_results)
saving_results[['whole_community_saving_bill_total_usd','whole_community_saving_emission_total_kgCO2e','whole_community_saving_energy_total_mmbtu','whole_community_saving_electricity_total_kwh','whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu']] = results

In [16]:
figure_total_savings_3(community_results) 

## 1.3 Annual Savings Per Dwelling Unit

In [17]:
results = figure_mean_savings(community_results) 
saving_results[['whole_community_saving_bill_mean_usd','whole_community_saving_emission_mean_kgCO2e','whole_community_saving_energy_mean_mmbtu']] = results

sample_weight


upgrade_name
Electric Clothes Dryer                                              14795.580898
Electric Cooking                                                    86222.523165
Induction Cooking                                                  352542.979330
Heat Pump Clothes Dryer                                            307646.044191
Heat Pump Water Heater                                             357134.711333
Mininum Efficiency Whole Home Electrification                      253820.741269
Mininum Efficiency Heat Pump with Electric Heat Backup             356879.615110
Mininum Efficiency Heat Pump with Existing Heat Backup             356114.326443
Basic Enclosure                                                    348716.535994
Enhanced Enclosure                                                 352032.786885
High Efficiency Heat Pump with Electric Heat Backup                356879.615110
High Efficiency Whole Home Electrification                         357900.000000
Basic Enclosure

In [18]:
metric = "pct_saving_energy.total_%"
print("Community summary:")
get_summary(community_results, metric, lower_percentile=0.25, upper_percentile=0.75)

Community summary:


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,1403,357900.000000,62.344996,54.2825,72.5600
Basic Enclosure + High Efficiency Whole Home Electrification,1403,357900.000000,61.692245,53.3525,71.9550
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1403,357900.000000,60.988004,52.5700,71.5475
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1403,357900.000000,60.335246,51.9075,70.6975
High Efficiency Whole Home Electrification,1403,357900.000000,56.831048,48.9500,66.3350
High Efficiency Heat Pump with Electric Heat Backup,1399,356879.615110,44.996040,34.9800,56.4450
Mininum Efficiency Whole Home Electrification,995,253820.741269,43.475176,38.8625,52.5050
Mininum Efficiency Heat Pump with Electric Heat Backup,1399,356879.615110,29.222352,18.0900,40.8225
Enhanced Enclosure,1380,352032.786885,24.148775,14.0500,33.3350


# 2. Results for Segments (All Income)

Segment Definition

Segment 1. Occupied, Single-Family Attached, 1940-1979, all income 

Segment 2. Occupied, Multifamily with 2-4 units, 1940-1979, all income 

Segment 3. Occupied, Single-Family Detached, 1940-1979, all income 

Segment 4. Occupied, Multifamily with 5+ units, 1940-1979, all income 


In [19]:
income = 'all_incomes'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Attached']
seg1_vintage = ['1940s', '1950s', '1960s', '1970s']
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg2_building_type = ['Multi-Family with 2 - 4 Units']
seg2_vintage = ['1940s', '1950s', '1960s', '1970s']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'

In [20]:
metric = 'pct_saving_energy.total_%'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    display(get_summary(df_seg, metric, lower_percentile=0.25, upper_percentile=0.75))

Summary for segment: segment1: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,12,3061.154669,66.225000,65.3350,75.2300
Basic Enclosure + High Efficiency Whole Home Electrification,12,3061.154669,64.852500,61.7650,74.5450
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,12,3061.154669,64.575833,64.0200,72.9950
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,12,3061.154669,63.202500,60.7100,72.3100
High Efficiency Whole Home Electrification,12,3061.154669,59.051667,50.3950,68.5400
Mininum Efficiency Whole Home Electrification,7,1785.673557,48.514286,48.4175,56.7275
High Efficiency Heat Pump with Electric Heat Backup,12,3061.154669,46.737500,41.1550,57.8750
Enhanced Enclosure,12,3061.154669,31.478333,20.8250,41.4250
Mininum Efficiency Heat Pump with Electric Heat Backup,12,3061.154669,30.940833,17.9100,42.0950


Summary for segment: segment2: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,54,13775.196009,60.549815,54.5300,69.0700
Basic Enclosure + High Efficiency Whole Home Electrification,54,13775.196009,59.876296,53.5800,68.5800
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,54,13775.196009,58.977037,53.3300,67.8200
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,54,13775.196009,58.303148,52.1000,67.6800
High Efficiency Whole Home Electrification,54,13775.196009,54.500000,47.1900,62.0500
High Efficiency Heat Pump with Electric Heat Backup,54,13775.196009,42.079259,33.2600,52.3000
Mininum Efficiency Whole Home Electrification,31,7907.982894,41.095161,34.7425,50.2125
Mininum Efficiency Heat Pump with Electric Heat Backup,54,13775.196009,26.948333,17.7000,36.5400
Enhanced Enclosure,54,13775.196009,22.868333,17.1300,28.8500


Summary for segment: segment3: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,372,94895.794726,67.735323,61.9850,74.150
Basic Enclosure + High Efficiency Whole Home Electrification,372,94895.794726,67.227366,61.5750,73.475
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,372,94895.794726,66.392796,60.6200,73.270
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,372,94895.794726,65.884946,60.3550,72.635
High Efficiency Whole Home Electrification,372,94895.794726,61.379140,57.0150,67.805
High Efficiency Heat Pump with Electric Heat Backup,372,94895.794726,50.689086,44.0350,58.790
Mininum Efficiency Whole Home Electrification,306,78059.444048,46.882026,43.1900,53.420
Mininum Efficiency Heat Pump with Electric Heat Backup,372,94895.794726,35.201237,28.2800,43.895
Enhanced Enclosure,372,94895.794726,30.837312,25.5400,36.055


Summary for segment: segment4: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,102,26019.814683,52.667745,44.4300,60.0600
Basic Enclosure + High Efficiency Whole Home Electrification,102,26019.814683,51.933137,44.0900,60.0600
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,102,26019.814683,50.856275,42.6300,58.0800
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,102,26019.814683,50.122157,42.6200,58.0200
High Efficiency Whole Home Electrification,102,26019.814683,47.555588,39.7400,53.3500
High Efficiency Heat Pump with Electric Heat Backup,102,26019.814683,31.940294,23.0100,41.4600
Mininum Efficiency Whole Home Electrification,54,13775.196009,29.618519,18.0700,41.8800
Mininum Efficiency Heat Pump with Electric Heat Backup,102,26019.814683,18.210000,8.2300,27.0700
Enhanced Enclosure,100,25509.622238,18.066000,11.3950,23.8750


## 2.1 Upgrade Cost Per Dwelling Unit 

In [21]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1000.8
Induction Cooking                                                   1720.9
Heat Pump Clothes Dryer                                             2261.5
Heat Pump Water Heater                                              2660.1
Basic Enclosure                                                     4954.4
Enhanced Enclosure                                                  6559.5
Mininum Efficiency Heat Pump with Existing Heat Backup             14398.1
Mininum Efficiency Heat Pump with Electric Heat Backup             14579.5
Mininum Efficiency Whole Home Electrification                      16527.0
High Efficiency Heat Pump with Electric Heat Backup                21541.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        26109.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     26396.3
High Efficiency Whole Home Electrification                         27807.2
Basic Enclos

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1000.8
Induction Cooking                                                   1720.9
Heat Pump Clothes Dryer                                             2261.5
Basic Enclosure                                                     2402.8
Heat Pump Water Heater                                              2579.0
Enhanced Enclosure                                                  3617.7
Mininum Efficiency Heat Pump with Existing Heat Backup             11862.5
Mininum Efficiency Heat Pump with Electric Heat Backup             11919.3
Mininum Efficiency Whole Home Electrification                      13849.7
High Efficiency Heat Pump with Electric Heat Backup                20530.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        23354.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     24270.4
High Efficiency Whole Home Electrification                         26174.6
Basic Enclos

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1000.8
Electric Clothes Dryer                                              1123.9
Induction Cooking                                                   1720.9
Heat Pump Clothes Dryer                                             2261.5
Heat Pump Water Heater                                              2853.4
Basic Enclosure                                                     7159.2
Enhanced Enclosure                                                  9807.1
Mininum Efficiency Heat Pump with Existing Heat Backup             17605.3
Mininum Efficiency Heat Pump with Electric Heat Backup             17695.6
Mininum Efficiency Whole Home Electrification                      20094.4
High Efficiency Heat Pump with Electric Heat Backup                26727.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        32366.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     33266.6
High Efficie

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1000.8
Basic Enclosure                                                     1534.9
Induction Cooking                                                   1720.9
Heat Pump Clothes Dryer                                             2261.5
Enhanced Enclosure                                                  2335.5
Heat Pump Water Heater                                              2588.6
Mininum Efficiency Whole Home Electrification                       9186.6
Mininum Efficiency Heat Pump with Electric Heat Backup             10151.7
Mininum Efficiency Heat Pump with Existing Heat Backup             10239.0
High Efficiency Heat Pump with Electric Heat Backup                18658.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        21338.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     21802.0
High Efficiency Whole Home Electrification                         24155.5
Basic Enclos

 ## 2.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [22]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.09969
Enhanced Enclosure                                                 0.09520
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04219
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04141
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03812
Basic Enclosure + High Efficiency Whole Home Electrification       0.03740
High Efficiency Heat Pump with Electric Heat Backup                0.03542
High Efficiency Whole Home Electrification                         0.03302
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02037
Heat Pump Water Heater                                             0.01901
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01779
Heat Pump Clothes Dryer                                            0.01195
Mininum Efficiency Whole Home Electrification                      0.00691
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.12647
Enhanced Enclosure                                                 0.09560
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03496
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03436
Heat Pump Water Heater                                             0.03180
Basic Enclosure + High Efficiency Whole Home Electrification       0.03179
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03136
High Efficiency Whole Home Electrification                         0.02721
High Efficiency Heat Pump with Electric Heat Backup                0.02654
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01815
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01583
Heat Pump Clothes Dryer                                            0.01279
Mininum Efficiency Whole Home Electrification                      0.00675
Induction Co

segment3


upgrade_name
Basic Enclosure                                                    0.07903
Enhanced Enclosure                                                 0.06876
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03886
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03847
Basic Enclosure + High Efficiency Whole Home Electrification       0.03565
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03538
Heat Pump Water Heater                                             0.03392
High Efficiency Heat Pump with Electric Heat Backup                0.03127
High Efficiency Whole Home Electrification                         0.03103
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01379
Heat Pump Clothes Dryer                                            0.01297
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00995
Mininum Efficiency Whole Home Electrification                      0.00742
Induction Co

segment4


upgrade_name
Basic Enclosure                                                    0.13577
Enhanced Enclosure                                                 0.12266
Heat Pump Water Heater                                             0.03493
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03119
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03069
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02824
Basic Enclosure + High Efficiency Whole Home Electrification       0.02774
High Efficiency Whole Home Electrification                         0.02473
High Efficiency Heat Pump with Electric Heat Backup                0.02373
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02015
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02004
Heat Pump Clothes Dryer                                            0.01127
Mininum Efficiency Whole Home Electrification                      0.00753
Induction Co

 ### Annual Energy Savings Per Dollar Invested

In [23]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.00518
Enhanced Enclosure                                                 0.00481
Mininum Efficiency Whole Home Electrification                      0.00338
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00241
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00240
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00225
High Efficiency Heat Pump with Electric Heat Backup                0.00222
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00216
Basic Enclosure + High Efficiency Whole Home Electrification       0.00215
High Efficiency Whole Home Electrification                         0.00209
Heat Pump Water Heater                                             0.00200
Electric Cooking                                                   0.00143
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00134
Heat Pump Cl

segment2


upgrade_name
Basic Enclosure                                                    0.00656
Enhanced Enclosure                                                 0.00492
Mininum Efficiency Whole Home Electrification                      0.00269
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00189
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00185
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00184
Basic Enclosure + High Efficiency Whole Home Electrification       0.00170
Heat Pump Water Heater                                             0.00169
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00166
High Efficiency Heat Pump with Electric Heat Backup                0.00159
High Efficiency Whole Home Electrification                         0.00157
Electric Cooking                                                   0.00139
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00126
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.00481
Enhanced Enclosure                                                 0.00415
Mininum Efficiency Whole Home Electrification                      0.00325
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00274
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00270
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00264
Heat Pump Water Heater                                             0.00262
High Efficiency Heat Pump with Electric Heat Backup                0.00256
Basic Enclosure + High Efficiency Whole Home Electrification       0.00245
High Efficiency Whole Home Electrification                         0.00242
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00241
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00170
Electric Cooking                                                   0.00155
Electric Clo

segment4


upgrade_name
Basic Enclosure                                                    0.00528
Enhanced Enclosure                                                 0.00463
Mininum Efficiency Whole Home Electrification                      0.00173
Heat Pump Water Heater                                             0.00166
Electric Cooking                                                   0.00142
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00126
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00125
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00114
Basic Enclosure + High Efficiency Whole Home Electrification       0.00113
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00108
High Efficiency Whole Home Electrification                         0.00105
High Efficiency Heat Pump with Electric Heat Backup                0.00097
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00095
Heat Pump Cl

 ### Annual Emission Savings Per Dollar Invested

In [24]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.43047
Enhanced Enclosure                                                 0.40555
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.19258
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.19034
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.17266
Basic Enclosure + High Efficiency Whole Home Electrification       0.17051
High Efficiency Heat Pump with Electric Heat Backup                0.16773
High Efficiency Whole Home Electrification                         0.15835
Mininum Efficiency Whole Home Electrification                      0.15690
Heat Pump Water Heater                                             0.14426
Mininum Efficiency Heat Pump with Electric Heat Backup             0.12961
Mininum Efficiency Heat Pump with Existing Heat Backup             0.10066
Electric Cooking                                                   0.05617
Heat Pump Cl

segment2


upgrade_name
Basic Enclosure                                                    0.54945
Enhanced Enclosure                                                 0.41391
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.15292
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.14963
Basic Enclosure + High Efficiency Whole Home Electrification       0.13793
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.13551
Heat Pump Water Heater                                             0.13183
High Efficiency Heat Pump with Electric Heat Backup                0.12333
Mininum Efficiency Whole Home Electrification                      0.12264
High Efficiency Whole Home Electrification                         0.12240
Mininum Efficiency Heat Pump with Electric Heat Backup             0.11072
Mininum Efficiency Heat Pump with Existing Heat Backup             0.09367
Electric Cooking                                                   0.05276
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.37092
Enhanced Enclosure                                                 0.32204
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.19506
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.19211
Heat Pump Water Heater                                             0.17945
Basic Enclosure + High Efficiency Whole Home Electrification       0.17725
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.17504
High Efficiency Heat Pump with Electric Heat Backup                0.17330
High Efficiency Whole Home Electrification                         0.16606
Mininum Efficiency Whole Home Electrification                      0.14856
Mininum Efficiency Heat Pump with Electric Heat Backup             0.13411
Mininum Efficiency Heat Pump with Existing Heat Backup             0.10264
Electric Cooking                                                   0.05845
Heat Pump Cl

segment4


upgrade_name
Basic Enclosure                                                    0.51867
Enhanced Enclosure                                                 0.46419
Heat Pump Water Heater                                             0.13391
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.11888
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.11732
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.10741
Basic Enclosure + High Efficiency Whole Home Electrification       0.10582
High Efficiency Whole Home Electrification                         0.09547
High Efficiency Heat Pump with Electric Heat Backup                0.09219
Mininum Efficiency Heat Pump with Electric Heat Backup             0.08883
Mininum Efficiency Heat Pump with Existing Heat Backup             0.08477
Mininum Efficiency Whole Home Electrification                      0.08456
Electric Cooking                                                   0.05650
Heat Pump Cl

## 2.3 Annual Total Savings

### Annual Total Bill Savings

In [25]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3500961.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3409331.2
Basic Enclosure + High Efficiency Whole Home Electrification       3401525.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3309894.9
High Efficiency Whole Home Electrification                         2811051.8
High Efficiency Heat Pump with Electric Heat Backup                2335419.0
Enhanced Enclosure                                                 1911624.5
Basic Enclosure                                                    1511922.2
Mininum Efficiency Heat Pump with Existing Heat Backup              897947.1
Mininum Efficiency Heat Pump with Electric Heat Backup              794050.8
Mininum Efficiency Whole Home Electrification                       203896.6
Heat Pump Water Heater                                              154793.3
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    11829871.2
Basic Enclosure + High Efficiency Whole Home Electrification       11589761.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     11486163.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        11246053.5
High Efficiency Whole Home Electrification                          9809960.9
High Efficiency Heat Pump with Electric Heat Backup                 7504919.7
Enhanced Enclosure                                                  4763990.2
Basic Enclosure                                                     4108612.8
Mininum Efficiency Heat Pump with Existing Heat Backup              2965307.7
Mininum Efficiency Heat Pump with Electric Heat Backup              2598625.8
Heat Pump Water Heater                                              1108868.3
Mininum Efficiency Whole Home Electrification                        738803.9
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    124909902.6
Basic Enclosure + High Efficiency Whole Home Electrification       122826253.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     121440025.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        119356376.5
High Efficiency Whole Home Electrification                          98686562.4
High Efficiency Heat Pump with Electric Heat Backup                 79311347.7
Enhanced Enclosure                                                  63991255.7
Basic Enclosure                                                     53687640.4
Mininum Efficiency Heat Pump with Existing Heat Backup              23041704.2
Mininum Efficiency Heat Pump with Electric Heat Backup              16702180.5
Mininum Efficiency Whole Home Electrification                       11640138.4
Heat Pump Water Heater                                               9159199.1
Heat Pump Clothes Dryer                

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    18175381.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     17692934.5
Basic Enclosure + High Efficiency Whole Home Electrification       17519860.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        17037413.2
High Efficiency Whole Home Electrification                         15544026.1
High Efficiency Heat Pump with Electric Heat Backup                11518831.5
Enhanced Enclosure                                                  7307682.3
Mininum Efficiency Heat Pump with Electric Heat Backup              5292410.9
Basic Enclosure                                                     5209977.6
Mininum Efficiency Heat Pump with Existing Heat Backup              5157162.7
Heat Pump Water Heater                                              2329555.1
Mininum Efficiency Whole Home Electrification                        952568.4
Heat Pump Clothes Dryer                            

### Annual Total Energy Savings

In [26]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    198639.1
Basic Enclosure + High Efficiency Whole Home Electrification       195654.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     194749.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        191764.5
High Efficiency Whole Home Electrification                         177927.6
High Efficiency Heat Pump with Electric Heat Backup                146551.0
Mininum Efficiency Heat Pump with Electric Heat Backup             100324.0
Mininum Efficiency Whole Home Electrification                       99659.2
Enhanced Enclosure                                                  96620.8
Basic Enclosure                                                     78631.6
Mininum Efficiency Heat Pump with Existing Heat Backup              59187.7
Heat Pump Water Heater                                              16292.5
Heat Pump Clothes Dryer                                              2069.9

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    627836.7
Basic Enclosure + High Efficiency Whole Home Electrification       620628.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     613732.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        606523.9
High Efficiency Whole Home Electrification                         567197.8
High Efficiency Heat Pump with Electric Heat Backup                450260.4
Mininum Efficiency Heat Pump with Electric Heat Backup             302985.9
Mininum Efficiency Whole Home Electrification                      294876.2
Enhanced Enclosure                                                 245417.1
Basic Enclosure                                                    213245.1
Mininum Efficiency Heat Pump with Existing Heat Backup             205675.7
Heat Pump Water Heater                                              58857.3
Heat Pump Clothes Dryer                                              7528.7

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    8494728.9
Basic Enclosure + High Efficiency Whole Home Electrification       8432176.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     8347029.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        8284476.9
High Efficiency Whole Home Electrification                         7707489.3
High Efficiency Heat Pump with Electric Heat Backup                6483326.6
Mininum Efficiency Whole Home Electrification                      5094302.2
Mininum Efficiency Heat Pump with Electric Heat Backup             4604423.6
Enhanced Enclosure                                                 3862975.4
Basic Enclosure                                                    3264683.7
Mininum Efficiency Heat Pump with Existing Heat Backup             2842940.5
Heat Pump Water Heater                                              707886.4
Heat Pump Clothes Dryer                                        

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    736212.8
Basic Enclosure + High Efficiency Whole Home Electrification       716533.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     714757.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        695078.8
High Efficiency Whole Home Electrification                         657217.7
High Efficiency Heat Pump with Electric Heat Backup                471258.1
Mininum Efficiency Heat Pump with Electric Heat Backup             284367.2
Enhanced Enclosure                                                 275976.1
Mininum Efficiency Heat Pump with Existing Heat Backup             243704.4
Mininum Efficiency Whole Home Electrification                      219174.1
Basic Enclosure                                                    202446.4
Heat Pump Water Heater                                             110437.0
Heat Pump Clothes Dryer                                             11122.7

### Annual Total Emission Savings

In [27]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    15857407.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     15561075.2
Basic Enclosure + High Efficiency Whole Home Electrification       15509622.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        15213290.1
High Efficiency Whole Home Electrification                         13479385.1
High Efficiency Heat Pump with Electric Heat Backup                11060633.3
Enhanced Enclosure                                                  8143266.2
Basic Enclosure                                                     6528639.7
Mininum Efficiency Heat Pump with Electric Heat Backup              5784612.6
Mininum Efficiency Whole Home Electrification                       4630423.6
Mininum Efficiency Heat Pump with Existing Heat Backup              4436648.9
Heat Pump Water Heater                                              1174697.5
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    51117315.1
Basic Enclosure + High Efficiency Whole Home Electrification       50286683.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     50026773.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        49196141.5
High Efficiency Whole Home Electrification                         44132347.7
High Efficiency Heat Pump with Electric Heat Backup                34879368.2
Enhanced Enclosure                                                 20627293.6
Mininum Efficiency Heat Pump with Electric Heat Backup             18178504.8
Basic Enclosure                                                    17849910.9
Mininum Efficiency Heat Pump with Existing Heat Backup             15305628.0
Mininum Efficiency Whole Home Electrification                      13431719.5
Heat Pump Water Heater                                              4596824.7
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    617977807.9
Basic Enclosure + High Efficiency Whole Home Electrification       610620363.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     606472769.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        599115324.4
High Efficiency Whole Home Electrification                         528072700.9
High Efficiency Heat Pump with Electric Heat Backup                439534045.0
Enhanced Enclosure                                                 299711822.7
Basic Enclosure                                                    251993509.9
Mininum Efficiency Whole Home Electrification                      233025090.8
Mininum Efficiency Heat Pump with Electric Heat Backup             225210796.4
Mininum Efficiency Heat Pump with Existing Heat Backup             171473538.3
Heat Pump Water Heater                                              48460461.1
Heat Pump Clothes Dryer                

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    69132438.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     67440034.9
Basic Enclosure + High Efficiency Whole Home Electrification       66833408.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        65141004.9
High Efficiency Whole Home Electrification                         60002883.2
High Efficiency Heat Pump with Electric Heat Backup                44755841.5
Enhanced Enclosure                                                 27655174.9
Mininum Efficiency Heat Pump with Electric Heat Backup             23463530.4
Mininum Efficiency Heat Pump with Existing Heat Backup             21698548.1
Basic Enclosure                                                    19902640.4
Mininum Efficiency Whole Home Electrification                      10700476.9
Heat Pump Water Heater                                              8931406.8
Heat Pump Clothes Dryer                            

## 2.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [28]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1143.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1113.7
Basic Enclosure + High Efficiency Whole Home Electrification       1111.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1081.3
High Efficiency Whole Home Electrification                          918.3
High Efficiency Heat Pump with Electric Heat Backup                 762.9
Enhanced Enclosure                                                  624.5
Basic Enclosure                                                     493.9
Mininum Efficiency Heat Pump with Existing Heat Backup              293.3
Mininum Efficiency Heat Pump with Electric Heat Backup              259.4
Mininum Efficiency Whole Home Electrification                       114.2
Heat Pump Water Heater                                               50.6
Heat Pump Clothes Dryer                                              27.0
Induction Cooking        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    858.8
Basic Enclosure + High Efficiency Whole Home Electrification       841.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     833.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        816.4
High Efficiency Whole Home Electrification                         712.1
High Efficiency Heat Pump with Electric Heat Backup                544.8
Enhanced Enclosure                                                 345.8
Basic Enclosure                                                    303.9
Mininum Efficiency Heat Pump with Existing Heat Backup             215.3
Mininum Efficiency Heat Pump with Electric Heat Backup             188.6
Mininum Efficiency Whole Home Electrification                       93.4
Heat Pump Water Heater                                              82.0
Heat Pump Clothes Dryer                                             28.9
Induction Cooking                     

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1316.3
Basic Enclosure + High Efficiency Whole Home Electrification       1294.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1279.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1257.8
High Efficiency Whole Home Electrification                         1039.9
High Efficiency Heat Pump with Electric Heat Backup                 835.8
Enhanced Enclosure                                                  674.3
Basic Enclosure                                                     565.8
Mininum Efficiency Heat Pump with Existing Heat Backup              242.8
Mininum Efficiency Heat Pump with Electric Heat Backup              176.0
Mininum Efficiency Whole Home Electrification                       149.1
Heat Pump Water Heater                                               96.8
Heat Pump Clothes Dryer                                              29.3
Induction Cooking        

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    698.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     680.0
Basic Enclosure + High Efficiency Whole Home Electrification       673.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        654.8
High Efficiency Whole Home Electrification                         597.4
High Efficiency Heat Pump with Electric Heat Backup                442.7
Enhanced Enclosure                                                 286.5
Basic Enclosure                                                    208.4
Mininum Efficiency Heat Pump with Existing Heat Backup             206.3
Mininum Efficiency Heat Pump with Electric Heat Backup             203.4
Heat Pump Water Heater                                              90.4
Mininum Efficiency Whole Home Electrification                       69.2
Heat Pump Clothes Dryer                                             25.5
Induction Cooking                     

### Annual Energy Savings Per Dwelling Unit

In [29]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    64.9
Basic Enclosure + High Efficiency Whole Home Electrification       63.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     63.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        62.6
High Efficiency Whole Home Electrification                         58.1
Mininum Efficiency Whole Home Electrification                      55.8
High Efficiency Heat Pump with Electric Heat Backup                47.9
Mininum Efficiency Heat Pump with Electric Heat Backup             32.8
Enhanced Enclosure                                                 31.6
Basic Enclosure                                                    25.7
Mininum Efficiency Heat Pump with Existing Heat Backup             19.3
Heat Pump Water Heater                                              5.3
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    45.6
Basic Enclosure + High Efficiency Whole Home Electrification       45.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     44.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        44.0
High Efficiency Whole Home Electrification                         41.2
Mininum Efficiency Whole Home Electrification                      37.3
High Efficiency Heat Pump with Electric Heat Backup                32.7
Mininum Efficiency Heat Pump with Electric Heat Backup             22.0
Enhanced Enclosure                                                 17.8
Basic Enclosure                                                    15.8
Mininum Efficiency Heat Pump with Existing Heat Backup             14.9
Heat Pump Water Heater                                              4.4
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    89.5
Basic Enclosure + High Efficiency Whole Home Electrification       88.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     88.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        87.3
High Efficiency Whole Home Electrification                         81.2
High Efficiency Heat Pump with Electric Heat Backup                68.3
Mininum Efficiency Whole Home Electrification                      65.3
Mininum Efficiency Heat Pump with Electric Heat Backup             48.5
Enhanced Enclosure                                                 40.7
Basic Enclosure                                                    34.4
Mininum Efficiency Heat Pump with Existing Heat Backup             30.0
Heat Pump Water Heater                                              7.5
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    28.3
Basic Enclosure + High Efficiency Whole Home Electrification       27.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     27.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        26.7
High Efficiency Whole Home Electrification                         25.3
High Efficiency Heat Pump with Electric Heat Backup                18.1
Mininum Efficiency Whole Home Electrification                      15.9
Mininum Efficiency Heat Pump with Electric Heat Backup             10.9
Enhanced Enclosure                                                 10.8
Mininum Efficiency Heat Pump with Existing Heat Backup              9.7
Basic Enclosure                                                     8.1
Heat Pump Water Heater                                              4.3
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

### Annual Percentage Energy Savings Per Dwelling Unit

In [30]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    66.2
Basic Enclosure + High Efficiency Whole Home Electrification       64.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     64.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        63.2
High Efficiency Whole Home Electrification                         59.1
Mininum Efficiency Whole Home Electrification                      48.5
High Efficiency Heat Pump with Electric Heat Backup                46.7
Enhanced Enclosure                                                 31.5
Mininum Efficiency Heat Pump with Electric Heat Backup             30.9
Basic Enclosure                                                    24.3
Mininum Efficiency Heat Pump with Existing Heat Backup             20.7
Heat Pump Water Heater                                              6.3
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    60.5
Basic Enclosure + High Efficiency Whole Home Electrification       59.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     59.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        58.3
High Efficiency Whole Home Electrification                         54.5
High Efficiency Heat Pump with Electric Heat Backup                42.1
Mininum Efficiency Whole Home Electrification                      41.1
Mininum Efficiency Heat Pump with Electric Heat Backup             26.9
Enhanced Enclosure                                                 22.9
Basic Enclosure                                                    20.4
Mininum Efficiency Heat Pump with Existing Heat Backup             20.1
Heat Pump Water Heater                                              6.7
Electric Cooking                                                    1.9
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    67.7
Basic Enclosure + High Efficiency Whole Home Electrification       67.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     66.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        65.9
High Efficiency Whole Home Electrification                         61.4
High Efficiency Heat Pump with Electric Heat Backup                50.7
Mininum Efficiency Whole Home Electrification                      46.9
Mininum Efficiency Heat Pump with Electric Heat Backup             35.2
Enhanced Enclosure                                                 30.8
Basic Enclosure                                                    25.5
Mininum Efficiency Heat Pump with Existing Heat Backup             22.6
Heat Pump Water Heater                                              6.2
Electric Cooking                                                    1.3
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    52.7
Basic Enclosure + High Efficiency Whole Home Electrification       51.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     50.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        50.1
High Efficiency Whole Home Electrification                         47.6
High Efficiency Heat Pump with Electric Heat Backup                31.9
Mininum Efficiency Whole Home Electrification                      29.6
Mininum Efficiency Heat Pump with Electric Heat Backup             18.2
Enhanced Enclosure                                                 18.1
Mininum Efficiency Heat Pump with Existing Heat Backup             15.7
Basic Enclosure                                                    15.2
Heat Pump Water Heater                                              9.5
Electric Cooking                                                    3.1
Heat Pump Clothes Dryer                            

### Annual Emission Savings Per Dwelling Unit

In [31]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5180.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5083.4
Basic Enclosure + High Efficiency Whole Home Electrification       5066.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4969.8
High Efficiency Whole Home Electrification                         4403.4
High Efficiency Heat Pump with Electric Heat Backup                3613.2
Enhanced Enclosure                                                 2660.2
Mininum Efficiency Whole Home Electrification                      2593.1
Basic Enclosure                                                    2132.7
Mininum Efficiency Heat Pump with Electric Heat Backup             1889.7
Mininum Efficiency Heat Pump with Existing Heat Backup             1449.3
Heat Pump Water Heater                                              383.7
Heat Pump Clothes Dryer                                              79.8
Electric Cooking         

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3710.8
Basic Enclosure + High Efficiency Whole Home Electrification       3650.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3631.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3571.4
High Efficiency Whole Home Electrification                         3203.8
High Efficiency Heat Pump with Electric Heat Backup                2532.0
Mininum Efficiency Whole Home Electrification                      1698.5
Enhanced Enclosure                                                 1497.4
Basic Enclosure                                                    1320.2
Mininum Efficiency Heat Pump with Electric Heat Backup             1319.7
Mininum Efficiency Heat Pump with Existing Heat Backup             1111.1
Heat Pump Water Heater                                              340.0
Heat Pump Clothes Dryer                                              85.3
Electric Cooking         

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    6512.2
Basic Enclosure + High Efficiency Whole Home Electrification       6434.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     6390.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        6313.4
High Efficiency Whole Home Electrification                         5564.8
High Efficiency Heat Pump with Electric Heat Backup                4631.8
Enhanced Enclosure                                                 3158.3
Mininum Efficiency Whole Home Electrification                      2985.2
Basic Enclosure                                                    2655.5
Mininum Efficiency Heat Pump with Electric Heat Backup             2373.2
Mininum Efficiency Heat Pump with Existing Heat Backup             1807.0
Heat Pump Water Heater                                              512.0
Heat Pump Clothes Dryer                                              92.3
Electric Cooking         

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2656.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2591.9
Basic Enclosure + High Efficiency Whole Home Electrification       2568.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2503.5
High Efficiency Whole Home Electrification                         2306.0
High Efficiency Heat Pump with Electric Heat Backup                1720.1
Enhanced Enclosure                                                 1084.1
Mininum Efficiency Heat Pump with Electric Heat Backup              901.8
Mininum Efficiency Heat Pump with Existing Heat Backup              868.0
Basic Enclosure                                                     796.1
Mininum Efficiency Whole Home Electrification                       776.8
Heat Pump Water Heater                                              346.7
Heat Pump Clothes Dryer                                              75.5
Electric Cooking         

## 2.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [32]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [33]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 3

In [34]:
segment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 4

In [35]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [36]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]

energy burden (%) before and after upgrade
segment1


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3555432078.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3555432078.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3555432078.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,9.9,5.2
Basic Enclosure + High Efficiency Whole Home Electrification,9.9,5.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,9.9,5.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,9.9,5.6
High Efficiency Whole Home Electrification,9.9,6.0
High Efficiency Heat Pump with Electric Heat Backup,9.9,7.2
Enhanced Enclosure,9.9,7.4
Basic Enclosure,9.9,8.2
Mininum Efficiency Whole Home Electrification,11.1,10.2


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,8.5,4.3
Basic Enclosure + High Efficiency Whole Home Electrification,8.5,4.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,8.5,4.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,8.5,4.5
High Efficiency Whole Home Electrification,8.5,5.0
High Efficiency Heat Pump with Electric Heat Backup,8.5,5.9
Enhanced Enclosure,8.5,6.8
Basic Enclosure,8.6,7.0
Mininum Efficiency Heat Pump with Existing Heat Backup,8.5,7.5


segment3


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,8.5,4.2
Basic Enclosure + High Efficiency Whole Home Electrification,8.5,4.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,8.5,4.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,8.5,4.4
High Efficiency Whole Home Electrification,8.5,5.2
High Efficiency Heat Pump with Electric Heat Backup,8.5,5.8
Enhanced Enclosure,8.5,6.2
Basic Enclosure,8.5,6.6
Mininum Efficiency Heat Pump with Existing Heat Backup,8.5,7.7


segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,10.1,5.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,10.1,5.2
Basic Enclosure + High Efficiency Whole Home Electrification,10.1,5.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,10.1,5.4
High Efficiency Whole Home Electrification,10.1,5.9
High Efficiency Heat Pump with Electric Heat Backup,10.1,6.8
Enhanced Enclosure,10.3,8.0
Mininum Efficiency Heat Pump with Existing Heat Backup,10.2,8.6
Mininum Efficiency Heat Pump with Electric Heat Backup,10.1,8.5


# 3. Results for Segments (0-80% AMI)

Segment Definition

Segment 1. Occupied, Single-Family Attached, 1940-1979, 0-80% AMI  

Segment 2. Occupied, Multifamily with 2-4 units, 1940-1979, 0-80% AMI  

Segment 3. Occupied, Single-Family Detached, 1940-1979, 0-80% AMI  

Segment 4. Occupied, Multifamily with 5+ units, 1940-1979, 0-80% AMI   

In [37]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")

income = 'specified_by_community'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Attached']
seg1_vintage = ['1940s', '1950s', '1960s', '1970s']
seg1_median_income = ['0-30%','30-60%','60-80%']

seg2_building_type = ['Multi-Family with 2 - 4 Units']
seg2_vintage = ['1940s', '1950s', '1960s', '1970s']
seg2_median_income = ['0-30%','30-60%','60-80%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_median_income = ['0-30%','30-60%','60-80%']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_median_income = ['0-30%','30-60%','60-80%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) &  
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'

## 3.1 Upgrade Cost Per Dwelling Unit 

In [38]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1000.8
Induction Cooking                                                   1720.9
Heat Pump Clothes Dryer                                             2261.5
Heat Pump Water Heater                                              2718.0
Basic Enclosure                                                     4706.6
Enhanced Enclosure                                                  6761.6
Mininum Efficiency Heat Pump with Existing Heat Backup             14917.8
Mininum Efficiency Heat Pump with Electric Heat Backup             14944.0
Mininum Efficiency Whole Home Electrification                      19162.2
High Efficiency Heat Pump with Electric Heat Backup                19387.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        24049.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     24308.1
High Efficiency Whole Home Electrification                         25765.3
Basic Enclos

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1000.8
Induction Cooking                                                   1720.9
Heat Pump Clothes Dryer                                             2261.5
Basic Enclosure                                                     2322.4
Heat Pump Water Heater                                              2579.0
Enhanced Enclosure                                                  3351.3
Mininum Efficiency Heat Pump with Existing Heat Backup             12467.0
Mininum Efficiency Heat Pump with Electric Heat Backup             12532.9
Mininum Efficiency Whole Home Electrification                      13510.8
High Efficiency Heat Pump with Electric Heat Backup                20161.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        22775.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     23493.6
High Efficiency Whole Home Electrification                         25867.0
Basic Enclos

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1000.8
Electric Clothes Dryer                                              1123.9
Induction Cooking                                                   1720.9
Heat Pump Clothes Dryer                                             2261.5
Heat Pump Water Heater                                              2807.5
Basic Enclosure                                                     6672.3
Enhanced Enclosure                                                  9301.6
Mininum Efficiency Heat Pump with Existing Heat Backup             16853.6
Mininum Efficiency Heat Pump with Electric Heat Backup             16939.7
Mininum Efficiency Whole Home Electrification                      19423.1
High Efficiency Heat Pump with Electric Heat Backup                25988.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        31282.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32137.3
High Efficie

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1000.8
Basic Enclosure                                                     1538.8
Induction Cooking                                                   1720.9
Heat Pump Clothes Dryer                                             2261.5
Enhanced Enclosure                                                  2374.2
Heat Pump Water Heater                                              2590.9
Mininum Efficiency Whole Home Electrification                       9045.2
Mininum Efficiency Heat Pump with Electric Heat Backup             10040.3
Mininum Efficiency Heat Pump with Existing Heat Backup             10146.4
High Efficiency Heat Pump with Electric Heat Backup                18898.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        21612.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     22093.9
High Efficiency Whole Home Electrification                         24370.4
Basic Enclos

 ## 3.2 Annual Savings Per Dollar Invested

### Annual Bill Savings Per Dollar Invested

In [39]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure                                                 0.07243
Basic Enclosure                                                    0.06656
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03383
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03236
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03068
Basic Enclosure + High Efficiency Whole Home Electrification       0.02938
High Efficiency Heat Pump with Electric Heat Backup                0.02733
High Efficiency Whole Home Electrification                         0.02658
Heat Pump Water Heater                                             0.01737
Heat Pump Clothes Dryer                                            0.01410
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01289
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01218
Mininum Efficiency Whole Home Electrification                      0.00778
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.12356
Enhanced Enclosure                                                 0.09419
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03421
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03366
Heat Pump Water Heater                                             0.03320
Basic Enclosure + High Efficiency Whole Home Electrification       0.03105
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03065
High Efficiency Whole Home Electrification                         0.02666
High Efficiency Heat Pump with Electric Heat Backup                0.02598
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01750
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01715
Heat Pump Clothes Dryer                                            0.01289
Mininum Efficiency Whole Home Electrification                      0.00544
Induction Co

segment3


upgrade_name
Basic Enclosure                                                    0.07970
Enhanced Enclosure                                                 0.06890
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03779
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03744
Basic Enclosure + High Efficiency Whole Home Electrification       0.03454
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03431
High Efficiency Heat Pump with Electric Heat Backup                0.03025
Heat Pump Water Heater                                             0.03011
High Efficiency Whole Home Electrification                         0.02981
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01369
Heat Pump Clothes Dryer                                            0.01223
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01022
Mininum Efficiency Whole Home Electrification                      0.00901
Induction Co

segment4


upgrade_name
Basic Enclosure                                                    0.13998
Enhanced Enclosure                                                 0.12807
Heat Pump Water Heater                                             0.03413
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03160
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03096
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02867
Basic Enclosure + High Efficiency Whole Home Electrification       0.02805
High Efficiency Whole Home Electrification                         0.02510
High Efficiency Heat Pump with Electric Heat Backup                0.02431
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02168
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02145
Heat Pump Clothes Dryer                                            0.01144
Mininum Efficiency Whole Home Electrification                      0.00892
Induction Co

 ### Annual Energy Savings Per Dollar Invested

In [40]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.00368
Enhanced Enclosure                                                 0.00367
Mininum Efficiency Whole Home Electrification                      0.00242
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00198
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00195
High Efficiency Heat Pump with Electric Heat Backup                0.00179
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00178
Basic Enclosure + High Efficiency Whole Home Electrification       0.00175
High Efficiency Whole Home Electrification                         0.00173
Heat Pump Water Heater                                             0.00164
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00163
Electric Cooking                                                   0.00140
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00122
Heat Pump Cl

segment2


upgrade_name
Basic Enclosure                                                    0.00583
Enhanced Enclosure                                                 0.00439
Mininum Efficiency Whole Home Electrification                      0.00210
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00167
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00163
Heat Pump Water Heater                                             0.00161
Basic Enclosure + High Efficiency Whole Home Electrification       0.00150
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00148
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00146
High Efficiency Whole Home Electrification                         0.00137
High Efficiency Heat Pump with Electric Heat Backup                0.00137
Electric Cooking                                                   0.00136
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00109
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.00485
Enhanced Enclosure                                                 0.00415
Mininum Efficiency Whole Home Electrification                      0.00315
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00268
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00261
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00256
Heat Pump Water Heater                                             0.00252
High Efficiency Heat Pump with Electric Heat Backup                0.00245
Basic Enclosure + High Efficiency Whole Home Electrification       0.00236
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00232
High Efficiency Whole Home Electrification                         0.00232
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00173
Electric Cooking                                                   0.00149
Electric Clo

segment4


upgrade_name
Basic Enclosure                                                    0.00540
Enhanced Enclosure                                                 0.00481
Mininum Efficiency Whole Home Electrification                      0.00184
Heat Pump Water Heater                                             0.00169
Electric Cooking                                                   0.00140
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00128
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00127
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00117
Basic Enclosure + High Efficiency Whole Home Electrification       0.00116
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00114
High Efficiency Whole Home Electrification                         0.00107
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00100
High Efficiency Heat Pump with Electric Heat Backup                0.00099
Heat Pump Cl

 ### Annual Emission Savings Per Dollar Invested

In [41]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure                                                 0.30986
Basic Enclosure                                                    0.29684
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.15530
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.15057
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.13911
Basic Enclosure + High Efficiency Whole Home Electrification       0.13485
High Efficiency Heat Pump with Electric Heat Backup                0.13345
High Efficiency Whole Home Electrification                         0.12783
Mininum Efficiency Whole Home Electrification                      0.11396
Heat Pump Water Heater                                             0.10611
Mininum Efficiency Heat Pump with Electric Heat Backup             0.09257
Mininum Efficiency Heat Pump with Existing Heat Backup             0.07868
Electric Cooking                                                   0.05642
Heat Pump Cl

segment2


upgrade_name
Basic Enclosure                                                    0.51273
Enhanced Enclosure                                                 0.38917
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.14204
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.13942
Heat Pump Water Heater                                             0.13075
Basic Enclosure + High Efficiency Whole Home Electrification       0.12796
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.12602
High Efficiency Heat Pump with Electric Heat Backup                0.11288
High Efficiency Whole Home Electrification                         0.11285
Mininum Efficiency Heat Pump with Electric Heat Backup             0.09803
Mininum Efficiency Whole Home Electrification                      0.09567
Mininum Efficiency Heat Pump with Existing Heat Backup             0.08498
Electric Cooking                                                   0.05293
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.37310
Enhanced Enclosure                                                 0.32175
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.18884
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.18614
Basic Enclosure + High Efficiency Whole Home Electrification       0.17092
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.16894
Heat Pump Water Heater                                             0.16891
High Efficiency Heat Pump with Electric Heat Backup                0.16628
High Efficiency Whole Home Electrification                         0.15883
Mininum Efficiency Whole Home Electrification                      0.14762
Mininum Efficiency Heat Pump with Electric Heat Backup             0.13139
Mininum Efficiency Heat Pump with Existing Heat Backup             0.10296
Electric Cooking                                                   0.05646
Heat Pump Cl

segment4


upgrade_name
Basic Enclosure                                                    0.53418
Enhanced Enclosure                                                 0.48423
Heat Pump Water Heater                                             0.13387
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.12095
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.11892
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.10952
Basic Enclosure + High Efficiency Whole Home Electrification       0.10751
High Efficiency Whole Home Electrification                         0.09739
Mininum Efficiency Heat Pump with Electric Heat Backup             0.09475
High Efficiency Heat Pump with Electric Heat Backup                0.09455
Mininum Efficiency Whole Home Electrification                      0.09247
Mininum Efficiency Heat Pump with Existing Heat Backup             0.09005
Electric Cooking                                                   0.05534
Heat Pump Cl

## 3.3 Annual Total Savings

### Annual Total Bill Savings

In [42]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1532184.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1468322.9
Basic Enclosure + High Efficiency Whole Home Electrification       1453456.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1389594.7
High Efficiency Whole Home Electrification                         1222786.4
High Efficiency Heat Pump with Electric Heat Backup                 946023.7
Enhanced Enclosure                                                  874462.9
Basic Enclosure                                                     559435.6
Mininum Efficiency Heat Pump with Existing Heat Backup              343435.1
Mininum Efficiency Heat Pump with Electric Heat Backup              325138.8
Mininum Efficiency Whole Home Electrification                       152211.0
Heat Pump Water Heater                                               84307.1
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    8117483.2
Basic Enclosure + High Efficiency Whole Home Electrification       8000975.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     7867500.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        7750992.7
High Efficiency Whole Home Electrification                         6861053.3
High Efficiency Heat Pump with Electric Heat Backup                5210938.6
Enhanced Enclosure                                                 3140502.0
Basic Enclosure                                                    2781640.6
Mininum Efficiency Heat Pump with Existing Heat Backup             2170407.1
Mininum Efficiency Heat Pump with Electric Heat Backup             2138238.0
Heat Pump Water Heater                                              851860.9
Mininum Efficiency Whole Home Electrification                       412737.6
Heat Pump Clothes Dryer                                        

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    62510912.9
Basic Enclosure + High Efficiency Whole Home Electrification       61443965.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     60769735.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        59702788.4
High Efficiency Whole Home Electrification                         49274182.7
High Efficiency Heat Pump with Electric Heat Backup                39711415.4
Enhanced Enclosure                                                 32371308.0
Basic Enclosure                                                    26860396.9
Mininum Efficiency Heat Pump with Existing Heat Backup             11651183.1
Mininum Efficiency Heat Pump with Electric Heat Backup              8747909.4
Mininum Efficiency Whole Home Electrification                       7363325.8
Heat Pump Water Heater                                              4248126.7
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    15181206.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     14784191.0
Basic Enclosure + High Efficiency Whole Home Electrification       14565520.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        14168505.5
High Efficiency Whole Home Electrification                         12952821.7
High Efficiency Heat Pump with Electric Heat Backup                 9728284.3
Enhanced Enclosure                                                  6360476.3
Mininum Efficiency Heat Pump with Electric Heat Backup              4560162.9
Mininum Efficiency Heat Pump with Existing Heat Backup              4432481.8
Basic Enclosure                                                     4395781.1
Heat Pump Water Heater                                              1849853.2
Mininum Efficiency Whole Home Electrification                        926686.6
Heat Pump Clothes Dryer                            

### Annual Total Energy Savings

In [43]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    88752.1
Basic Enclosure + High Efficiency Whole Home Electrification       86388.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     86007.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        83644.0
High Efficiency Whole Home Electrification                         79463.7
High Efficiency Heat Pump with Electric Heat Backup                62111.8
Mininum Efficiency Whole Home Electrification                      47324.4
Enhanced Enclosure                                                 44296.9
Mininum Efficiency Heat Pump with Electric Heat Backup             43491.6
Mininum Efficiency Heat Pump with Existing Heat Backup             32479.4
Basic Enclosure                                                    30891.6
Heat Pump Water Heater                                              7983.2
Heat Pump Clothes Dryer                                             1465.0
Induction Co

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    390838.3
Basic Enclosure + High Efficiency Whole Home Electrification       387340.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     380959.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        377461.8
High Efficiency Whole Home Electrification                         353119.5
High Efficiency Heat Pump with Electric Heat Backup                274049.1
Mininum Efficiency Heat Pump with Electric Heat Backup             181818.6
Mininum Efficiency Whole Home Electrification                      159537.2
Enhanced Enclosure                                                 146448.4
Mininum Efficiency Heat Pump with Existing Heat Backup             135120.1
Basic Enclosure                                                    131148.0
Heat Pump Water Heater                                              41327.9
Heat Pump Clothes Dryer                                              5582.3

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4231735.1
Basic Enclosure + High Efficiency Whole Home Electrification       4199705.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4159278.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4127248.7
High Efficiency Whole Home Electrification                         3834360.5
High Efficiency Heat Pump with Electric Heat Backup                3218307.7
Mininum Efficiency Whole Home Electrification                      2576171.3
Mininum Efficiency Heat Pump with Electric Heat Backup             2289136.6
Enhanced Enclosure                                                 1951440.9
Basic Enclosure                                                    1634673.9
Mininum Efficiency Heat Pump with Existing Heat Backup             1475968.6
Heat Pump Water Heater                                              356165.9
Heat Pump Clothes Dryer                                        

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    618830.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     600765.4
Basic Enclosure + High Efficiency Whole Home Electrification       600346.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        582282.1
High Efficiency Whole Home Electrification                         551932.6
High Efficiency Heat Pump with Electric Heat Backup                397702.2
Mininum Efficiency Heat Pump with Electric Heat Backup             242582.2
Enhanced Enclosure                                                 238873.9
Mininum Efficiency Heat Pump with Existing Heat Backup             203765.3
Mininum Efficiency Whole Home Electrification                      190901.3
Basic Enclosure                                                    169710.9
Heat Pump Water Heater                                              91807.3
Heat Pump Clothes Dryer                                              9110.8

### Annual Total Emission Savings

In [44]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    6947070.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     6741109.5
Basic Enclosure + High Efficiency Whole Home Electrification       6672216.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        6466255.6
High Efficiency Whole Home Electrification                         5881204.9
High Efficiency Heat Pump with Electric Heat Backup                4619970.8
Enhanced Enclosure                                                 3741260.8
Basic Enclosure                                                    2494805.2
Mininum Efficiency Heat Pump with Electric Heat Backup             2470186.0
Mininum Efficiency Whole Home Electrification                      2228208.3
Mininum Efficiency Heat Pump with Existing Heat Backup             2095837.1
Heat Pump Water Heater                                              515022.0
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    33375453.2
Basic Enclosure + High Efficiency Whole Home Electrification       32973564.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32587233.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        32185344.7
High Efficiency Whole Home Electrification                         29040343.4
High Efficiency Heat Pump with Electric Heat Backup                22640751.0
Enhanced Enclosure                                                 12975108.2
Mininum Efficiency Heat Pump with Electric Heat Backup             12223110.6
Basic Enclosure                                                    11543049.8
Mininum Efficiency Heat Pump with Existing Heat Backup             10539937.2
Mininum Efficiency Whole Home Electrification                       7253895.6
Heat Pump Water Heater                                              3354694.7
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    307834135.4
Basic Enclosure + High Efficiency Whole Home Electrification       304070310.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     302140381.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        298376555.6
High Efficiency Whole Home Electrification                         262497567.2
High Efficiency Heat Pump with Electric Heat Backup                218267038.0
Enhanced Enclosure                                                 151161530.4
Basic Enclosure                                                    125738539.6
Mininum Efficiency Whole Home Electrification                      120687564.3
Mininum Efficiency Heat Pump with Electric Heat Backup             112421531.4
Mininum Efficiency Heat Pump with Existing Heat Backup              87647427.8
Heat Pump Water Heater                                              23830445.2
Heat Pump Clothes Dryer                

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    57987673.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     56581892.6
Basic Enclosure + High Efficiency Whole Home Electrification       55822874.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        54417093.8
High Efficiency Whole Home Electrification                         50251729.8
High Efficiency Heat Pump with Electric Heat Backup                37832929.1
Enhanced Enclosure                                                 24048393.1
Mininum Efficiency Heat Pump with Electric Heat Backup             20142628.9
Mininum Efficiency Heat Pump with Existing Heat Backup             18413221.1
Basic Enclosure                                                    16775459.3
Mininum Efficiency Whole Home Electrification                       9601744.4
Heat Pump Water Heater                                              7254894.0
Heat Pump Clothes Dryer                            

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3926536263.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



## 3.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [45]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    858.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     822.3
Basic Enclosure + High Efficiency Whole Home Electrification       814.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        778.2
High Efficiency Whole Home Electrification                         684.8
High Efficiency Heat Pump with Electric Heat Backup                529.8
Enhanced Enclosure                                                 489.7
Basic Enclosure                                                    313.3
Mininum Efficiency Heat Pump with Existing Heat Backup             192.3
Mininum Efficiency Heat Pump with Electric Heat Backup             182.1
Mininum Efficiency Whole Home Electrification                      149.2
Heat Pump Water Heater                                              47.2
Heat Pump Clothes Dryer                                             31.9
Induction Cooking                     

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    815.9
Basic Enclosure + High Efficiency Whole Home Electrification       804.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     790.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        779.1
High Efficiency Whole Home Electrification                         689.6
High Efficiency Heat Pump with Electric Heat Backup                523.8
Enhanced Enclosure                                                 315.7
Basic Enclosure                                                    287.0
Mininum Efficiency Heat Pump with Existing Heat Backup             218.2
Mininum Efficiency Heat Pump with Electric Heat Backup             214.9
Heat Pump Water Heater                                              85.6
Mininum Efficiency Whole Home Electrification                       73.5
Heat Pump Clothes Dryer                                             29.2
Induction Cooking                     

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1237.6
Basic Enclosure + High Efficiency Whole Home Electrification       1216.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1203.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1182.0
High Efficiency Whole Home Electrification                          975.6
High Efficiency Heat Pump with Electric Heat Backup                 786.2
Enhanced Enclosure                                                  640.9
Basic Enclosure                                                     531.8
Mininum Efficiency Heat Pump with Existing Heat Backup              230.7
Mininum Efficiency Whole Home Electrification                       174.9
Mininum Efficiency Heat Pump with Electric Heat Backup              173.2
Heat Pump Water Heater                                               84.5
Heat Pump Clothes Dryer                                              27.7
Induction Cooking        

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    717.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     698.3
Basic Enclosure + High Efficiency Whole Home Electrification       687.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        669.2
High Efficiency Whole Home Electrification                         611.8
High Efficiency Heat Pump with Electric Heat Backup                459.5
Enhanced Enclosure                                                 304.1
Mininum Efficiency Heat Pump with Existing Heat Backup             219.9
Basic Enclosure                                                    215.4
Mininum Efficiency Heat Pump with Electric Heat Backup             215.4
Heat Pump Water Heater                                              88.4
Mininum Efficiency Whole Home Electrification                       80.7
Heat Pump Clothes Dryer                                             25.9
Induction Cooking                     

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3071005957.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3071005957.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3071005957.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Annual Energy Savings Per Dwelling Unit

In [46]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    49.7
Basic Enclosure + High Efficiency Whole Home Electrification       48.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     48.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        46.8
Mininum Efficiency Whole Home Electrification                      46.4
High Efficiency Whole Home Electrification                         44.5
High Efficiency Heat Pump with Electric Heat Backup                34.8
Enhanced Enclosure                                                 24.8
Mininum Efficiency Heat Pump with Electric Heat Backup             24.4
Mininum Efficiency Heat Pump with Existing Heat Backup             18.2
Basic Enclosure                                                    17.3
Heat Pump Water Heater                                              4.5
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    39.3
Basic Enclosure + High Efficiency Whole Home Electrification       38.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     38.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        37.9
High Efficiency Whole Home Electrification                         35.5
Mininum Efficiency Whole Home Electrification                      28.4
High Efficiency Heat Pump with Electric Heat Backup                27.5
Mininum Efficiency Heat Pump with Electric Heat Backup             18.3
Enhanced Enclosure                                                 14.7
Mininum Efficiency Heat Pump with Existing Heat Backup             13.6
Basic Enclosure                                                    13.5
Heat Pump Water Heater                                              4.2
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    83.8
Basic Enclosure + High Efficiency Whole Home Electrification       83.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     82.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        81.7
High Efficiency Whole Home Electrification                         75.9
High Efficiency Heat Pump with Electric Heat Backup                63.7
Mininum Efficiency Whole Home Electrification                      61.2
Mininum Efficiency Heat Pump with Electric Heat Backup             45.3
Enhanced Enclosure                                                 38.6
Basic Enclosure                                                    32.4
Mininum Efficiency Heat Pump with Existing Heat Backup             29.2
Heat Pump Water Heater                                              7.1
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    29.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     28.4
Basic Enclosure + High Efficiency Whole Home Electrification       28.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        27.5
High Efficiency Whole Home Electrification                         26.1
High Efficiency Heat Pump with Electric Heat Backup                18.8
Mininum Efficiency Whole Home Electrification                      16.6
Mininum Efficiency Heat Pump with Electric Heat Backup             11.5
Enhanced Enclosure                                                 11.4
Mininum Efficiency Heat Pump with Existing Heat Backup             10.1
Basic Enclosure                                                     8.3
Heat Pump Water Heater                                              4.4
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/396568384.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/396568384.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/396568384.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

### Annual Percentage Energy Savings Per Dwelling Unit

In [47]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    65.9
Basic Enclosure + High Efficiency Whole Home Electrification       64.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     63.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        61.8
High Efficiency Whole Home Electrification                         58.8
Mininum Efficiency Whole Home Electrification                      53.6
High Efficiency Heat Pump with Electric Heat Backup                44.3
Enhanced Enclosure                                                 31.4
Mininum Efficiency Heat Pump with Electric Heat Backup             29.8
Mininum Efficiency Heat Pump with Existing Heat Backup             22.7
Basic Enclosure                                                    21.7
Heat Pump Water Heater                                              6.9
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    60.1
Basic Enclosure + High Efficiency Whole Home Electrification       59.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     58.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        57.9
High Efficiency Whole Home Electrification                         54.0
High Efficiency Heat Pump with Electric Heat Backup                41.1
Mininum Efficiency Whole Home Electrification                      38.1
Mininum Efficiency Heat Pump with Electric Heat Backup             26.2
Enhanced Enclosure                                                 22.7
Basic Enclosure                                                    20.7
Mininum Efficiency Heat Pump with Existing Heat Backup             19.9
Heat Pump Water Heater                                              6.9
Electric Cooking                                                    2.1
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    67.8
Basic Enclosure + High Efficiency Whole Home Electrification       67.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     66.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        66.0
High Efficiency Whole Home Electrification                         61.4
High Efficiency Heat Pump with Electric Heat Backup                50.5
Mininum Efficiency Whole Home Electrification                      47.2
Mininum Efficiency Heat Pump with Electric Heat Backup             35.1
Enhanced Enclosure                                                 31.4
Basic Enclosure                                                    25.8
Mininum Efficiency Heat Pump with Existing Heat Backup             23.2
Heat Pump Water Heater                                              6.4
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    53.4
Basic Enclosure + High Efficiency Whole Home Electrification       52.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     51.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        50.8
High Efficiency Whole Home Electrification                         48.2
High Efficiency Heat Pump with Electric Heat Backup                32.4
Mininum Efficiency Whole Home Electrification                      30.6
Mininum Efficiency Heat Pump with Electric Heat Backup             18.6
Enhanced Enclosure                                                 18.4
Mininum Efficiency Heat Pump with Existing Heat Backup             15.7
Basic Enclosure                                                    15.5
Heat Pump Water Heater                                              9.7
Electric Cooking                                                    3.0
Heat Pump Clothes Dryer                            

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/1358844843.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/1358844843.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/1358844843.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Annual Emission Savings Per Dwelling Unit

In [48]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3890.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3775.1
Basic Enclosure + High Efficiency Whole Home Electrification       3736.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3621.2
High Efficiency Whole Home Electrification                         3293.5
High Efficiency Heat Pump with Electric Heat Backup                2587.2
Mininum Efficiency Whole Home Electrification                      2183.7
Enhanced Enclosure                                                 2095.2
Basic Enclosure                                                    1397.1
Mininum Efficiency Heat Pump with Electric Heat Backup             1383.3
Mininum Efficiency Heat Pump with Existing Heat Backup             1173.7
Heat Pump Water Heater                                              288.4
Heat Pump Clothes Dryer                                              93.2
Electric Cooking         

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3354.7
Basic Enclosure + High Efficiency Whole Home Electrification       3314.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3275.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3235.1
High Efficiency Whole Home Electrification                         2919.0
High Efficiency Heat Pump with Electric Heat Backup                2275.7
Enhanced Enclosure                                                 1304.2
Mininum Efficiency Whole Home Electrification                      1292.5
Mininum Efficiency Heat Pump with Electric Heat Backup             1228.6
Basic Enclosure                                                    1190.8
Mininum Efficiency Heat Pump with Existing Heat Backup             1059.4
Heat Pump Water Heater                                              337.2
Heat Pump Clothes Dryer                                              86.3
Electric Cooking         

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    6094.6
Basic Enclosure + High Efficiency Whole Home Electrification       6020.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5981.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5907.4
High Efficiency Whole Home Electrification                         5197.0
High Efficiency Heat Pump with Electric Heat Backup                4321.3
Enhanced Enclosure                                                 2992.8
Mininum Efficiency Whole Home Electrification                      2867.3
Basic Enclosure                                                    2489.4
Mininum Efficiency Heat Pump with Electric Heat Backup             2225.8
Mininum Efficiency Heat Pump with Existing Heat Backup             1735.3
Heat Pump Water Heater                                              474.2
Heat Pump Clothes Dryer                                              85.6
Electric Cooking         

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2738.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2672.4
Basic Enclosure + High Efficiency Whole Home Electrification       2636.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2570.1
High Efficiency Whole Home Electrification                         2373.4
High Efficiency Heat Pump with Electric Heat Backup                1786.8
Enhanced Enclosure                                                 1149.7
Mininum Efficiency Heat Pump with Electric Heat Backup              951.3
Mininum Efficiency Heat Pump with Existing Heat Backup              913.7
Mininum Efficiency Whole Home Electrification                       836.4
Basic Enclosure                                                     822.0
Heat Pump Water Heater                                              346.8
Heat Pump Clothes Dryer                                              76.5
Electric Cooking         

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/624873452.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/624873452.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/624873452.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

## 3.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [49]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/868763001.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/868763001.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/868763001.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

### Segment 2

In [50]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3209730133.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3209730133.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3209730133.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Segment 3

In [51]:
egment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/1326448381.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/1326448381.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/1326448381.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Segment 4

In [52]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/2289506022.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/2289506022.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/2289506022.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

## 3.6. Average Energy Burden (%) Before and after Upgrade

In [53]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]

energy burden (%) before and after upgrade
segment1


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3555432078.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3555432078.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/3555432078.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,14.3,7.8
Basic Enclosure + High Efficiency Whole Home Electrification,14.3,8.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,14.3,8.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,14.3,8.6
High Efficiency Whole Home Electrification,14.3,9.0
Enhanced Enclosure,14.3,10.8
High Efficiency Heat Pump with Electric Heat Backup,14.3,10.9
Basic Enclosure,14.3,12.2
Mininum Efficiency Whole Home Electrification,16.8,15.3


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,10.6,5.3
Basic Enclosure + High Efficiency Whole Home Electrification,10.6,5.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,10.6,5.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,10.6,5.5
High Efficiency Whole Home Electrification,10.6,6.3
High Efficiency Heat Pump with Electric Heat Backup,10.6,7.3
Enhanced Enclosure,10.6,8.5
Basic Enclosure,10.8,8.8
Mininum Efficiency Heat Pump with Existing Heat Backup,10.6,9.4


segment3


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,12.5,6.3
Basic Enclosure + High Efficiency Whole Home Electrification,12.5,6.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,12.5,6.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,12.5,6.5
High Efficiency Whole Home Electrification,12.5,7.6
High Efficiency Heat Pump with Electric Heat Backup,12.5,8.6
Enhanced Enclosure,12.5,9.2
Basic Enclosure,12.5,9.8
Mininum Efficiency Heat Pump with Existing Heat Backup,12.5,11.4


segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,11.9,6.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,11.9,6.1
Basic Enclosure + High Efficiency Whole Home Electrification,11.9,6.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,11.9,6.4
High Efficiency Whole Home Electrification,11.9,6.9
High Efficiency Heat Pump with Electric Heat Backup,11.9,8.1
Enhanced Enclosure,12.0,9.4
Mininum Efficiency Heat Pump with Existing Heat Backup,12.2,10.2
Mininum Efficiency Heat Pump with Electric Heat Backup,11.9,10.0


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/2330438264.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/2330438264.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_93509/2330438264.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [54]:
saving_results.to_csv(fig_dir / f'saving_results_{community}.csv')

### Save html file

In [55]:
! jupyter nbconvert --output-dir='./results/louisville/' --to=html --no-input saving_results_reports_for_louisville.ipynb

[NbConvertApp] Converting notebook saving_results_reports_for_louisville.ipynb to html
/Users/lliu2/micromamba/envs/res-est/share/jupyter/nbconvert/templates/base/display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  {%- elif type == 'application/javascript' -%}
[NbConvertApp] Writing 499578 bytes to results/louisville/saving_results_reports_for_louisville.html
